https://towardsdatascience.com/supply-planning-using-linear-programming-with-python-bff2401bf270

In [56]:
from pulp import  LpProblem, LpVariable, LpMinimize, lpSum, LpMinimize, LpStatus, value
import pandas as pd
import numpy as np

In [7]:
# 1. Initiliaze Class
model = LpProblem("Transhipment_Problem", LpMinimize)


In [8]:

# 2. Define Decision Variables
# Inbound Flows
I = LpVariable.dicts("I", [(i+1,j+1) for i in range(2) for j in range(2)],
                     lowBound=0, upBound=None, cat='Integer') # I(i,j) from plant i for platform j
# Outbound Flows
O = LpVariable.dicts("O", [(i+1,j+1) for i in range(2) for j in range(200)],
                     lowBound=0, upBound=None, cat='Integer') # O(i,j) from platform i for customer j


In [32]:
df_inbound = pd.read_csv('df_inbound_price.csv')

In [37]:
df_outbound = pd.read_csv('df_outboundprice.csv')
df_outbound.drop(columns=['Unnamed: 0'], inplace=True)

In [38]:
# 3. Define Objective Function
# Total Transportation Cost
model += lpSum([df_inbound.iloc[i,j+1] * I[i+1,j+1] for i in range(2) for j in range(2)]) + lpSum([df_outbound.iloc[i,j+1] * O[i+1,j+1] for i in range(2) for j in range(200)]) 


In [50]:
data = np.random.randint(1, 11, size=(200, 1))
df_demand = pd.DataFrame(data, columns=['DEMAND'])

In [54]:
# 4. Define Constraints
# Max capacity for plants
# Supply >= demand per store
for j in range(200):
    model += lpSum([O[i+1, j+1] for i in range(2)]) >= df_demand.loc[j,'DEMAND']
# Conservation of the flow in the local DC (X-Docking Platform)
for p in range(2):
    model += lpSum([I[i+1, p+1] for i in range(2)]) == lpSum([O[p+1, j+1] for j in range(200)])


In [57]:
# Solve Model
status = model.solve()
print(LpStatus[status])
print("Objective: z* = {}".format(
    value(model.objective)))

# Matrix result
inbound, outbound = np.zeros([2,2]), np.zeros([2,200])
for i in range(2):
    for j in range(2):
#         print(I[i+1, j+1].varValue, I[i+1, j+1].name)
        inbound[i, j] = I[i+1, j+1].varValue
for i in range(2):
    for j in range(200):
#         print(O[i+1, j+1].varValue, O[i+1, j+1].name)
        outbound[i, j] = O[i+1, j+1].varValue

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mdob/playground/ADDITIONAL/Various-tools-and-notes/vs_scripts3.12/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/aa280f9c4afd412ba261dd40718fc90f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/aa280f9c4afd412ba261dd40718fc90f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 207 COLUMNS
At line 2224 RHS
At line 2427 BOUNDS
At line 2832 ENDATA
Problem MODEL has 202 rows, 404 columns and 804 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7228.74 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 7228.74 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rou

In [58]:
inbound

array([[  0., 197.],
       [883.,   0.]])

In [59]:
outbound

array([[ 3.,  8.,  1.,  9.,  4.,  1.,  0.,  9., 10.,  0.,  9.,  4.,  6.,
         0.,  0.,  8., 10.,  2., 10.,  5.,  5.,  6., 10.,  2.,  7.,  5.,
         1.,  4.,  0.,  5.,  2.,  8.,  5.,  8.,  0.,  6.,  7.,  1.,  1.,
         5.,  0.,  8.,  9.,  1.,  9.,  1.,  7.,  2.,  1.,  6.,  5.,  9.,
         1.,  1., 10., 10.,  4.,  8.,  4., 10.,  9.,  6.,  9.,  2., 10.,
         0.,  8.,  8.,  0.,  4.,  7.,  8.,  0.,  1.,  1.,  1.,  6.,  0.,
         7.,  0.,  3.,  8.,  1., 10.,  8.,  0.,  3., 10.,  6.,  0.,  9.,
         1.,  4.,  9.,  0.,  0.,  2.,  8.,  4.,  4., 10.,  0.,  9.,  0.,
        10.,  0.,  4.,  1.,  2., 10.,  3.,  2.,  2.,  4.,  3.,  2.,  1.,
         0.,  6.,  0.,  2.,  7.,  5.,  2.,  6.,  9.,  9.,  7.,  7.,  0.,
         6.,  4.,  4.,  7.,  5.,  0.,  5.,  1.,  5.,  0.,  7.,  3.,  0.,
         5.,  2.,  1.,  0.,  1.,  5.,  5.,  0.,  0.,  0.,  8.,  6.,  4.,
         8.,  1.,  0.,  6.,  7.,  4.,  0.,  0.,  6.,  4.,  0.,  8.,  3.,
         8.,  8.,  4.,  9.,  6.,  1.,  6., 10.,  4.